In [1]:
%load_ext autoreload
%autoreload 2

In [1]:

import numpy as np
import AutoRecRecommender,AutoRecDataPrep2
import pandas as pd
import torch
from sklearn import preprocessing
from time import perf_counter

In [8]:
a = torch.ones(10).reshape((5,2))
a[torch.tensor([1,2,3,3,3,3])]

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

In [31]:
start = perf_counter()
a = torch.arange(20000).reshape((200,100))
print(a[[1,1,1]],a[[2,2,2]])
torch.sum(a[list(range(150))]*(a[list(range(50,200))]),dim=1)
print((perf_counter()-start)*800000)

tensor([[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
         114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
         128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
         142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
         156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
         170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
         184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197,
         198, 199],
        [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
         114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
         128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
         142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
         156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
         170, 171, 172, 173, 174

In [4]:
def get_time_array(data:pd.DataFrame):
    groups = data.loc[:,["timestamp","userId"]].groupby("userId")
    print("Done grouping")
    groupmins = groups["timestamp"].apply(pd.Series.min)
    times = data.loc[:,["timestamp"]].to_numpy(dtype=np.float32).reshape((-1,1))
    print("Done finding min",len(groupmins))
    for name in groups.indices.keys():
        # times[groups.indices[name]]= preprocessing.normalize([times[groups.indices[name]]])
        min_max_scaler = preprocessing.MinMaxScaler()
        times[groups.indices[name]] = min_max_scaler.fit_transform(times[groups.indices[name]])
    data.loc[:,["timestamp"]] = pd.Series(times.flatten())
    # print("------------------")
    # print(data.groupby("userId").min(["timestamp"]))
    # print("----------------------")
    # print(data.groupby("userId").max(["timestamp"]))
    # print("----------------------")
    # print(data.head())

In [13]:
a = pd.read_feather(r"cleaned_final_data.feather")
# a = pd.read_csv(r"/content/NCF/ratings.csv")
a.head()

,username,anime_id,score,timestamp
0,126070,241,6,1293559897
1,126070,650,8,1293560393
2,126070,8074,9,1293560058
3,126070,142,6,1294080780
4,126070,252,10,1293559649


In [6]:
print(a.dtypes)
print(a.shape)

username     int32
anime_id     int32
score         int8
timestamp    int32
dtype: object
(57233503, 4)


In [16]:
# a.drop(columns=["timestamp"],inplace=True)
a = a.to_numpy().astype("int32")

In [19]:
a[0]

array([    126070,        241,          6, 1293559897])

In [14]:
device = torch.device("cpu")
if torch.cuda.is_available(): 
  print("we have cuda!")
  device = torch.device("cuda:0")

In [20]:
train = AutoRecDataPrep2.MovieLensTrainDataloader2(data=a,batch_size=2048)

Times stats 1660972384 0
Rescaled item numbers
Made sparse matrix


In [21]:
train.sparse_ratings2.device

device(type='cpu')

In [22]:
train.sparse_ratings2.shape

torch.Size([593067, 3494])

In [30]:
model = AutoRecRecommender.AutoRec(train.n_users, train.n_items,epochs= 30, hidden_size= 40)
model.to(device)

AutoRec()

In [24]:
torch.cuda.empty_cache()

In [36]:
model.fit(train)

Predictions: N(3.852795362472534,2.367318868637085)
Labels: N(3.8567678928375244,0.8026875257492065)
-----------------------------------------------
Current Train RMSE: 2.464524984359741; Current Test RMSE: 1.0 Best Test RMSE: 1.0
Predictions: N(3.857240676879883,1.1938449144363403)
Labels: N(3.8567676544189453,0.8026875257492065)
-----------------------------------------------
Current Train RMSE: 1.3425321578979492; Current Test RMSE: 1.0 Best Test RMSE: 1.0
Predictions: N(3.8579485416412354,0.9588192701339722)
Labels: N(3.856767416000366,0.8026875257492065)
-----------------------------------------------
Current Train RMSE: 1.122326135635376; Current Test RMSE: 1.0 Best Test RMSE: 1.0
Predictions: N(3.8569343090057373,0.8359776139259338)
Labels: N(3.8567676544189453,0.8026875257492065)
-----------------------------------------------
Current Train RMSE: 1.0036529302597046; Current Test RMSE: 1.0 Best Test RMSE: 1.0
Predictions: N(3.8572065830230713,0.752622663974762)
Labels: N(3.85676

1.0

In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity
with profile(activities=[ProfilerActivity.CPU,ProfilerActivity.CUDA]) as prof:
    with record_function("model_inference"):
        model.fit(train)
        # for input in iter(train):
        #   pass

In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

In [ ]:
test = AutoRecTestDataPrep.MovieLensTestDataloader(train_data,test_data,batch_size=4096)
print(test.n_users,test.n_items)
model.eval()
model.evaluate(test)

897845 4315
Predictions: N(3.0860843658447266,0.7042092680931091)
Labels: N(3.621346950531006,0.8693665862083435)
Test RMSE: 0.9915255904197693


In [ ]:
torch.save(model.state_dict(), "movielense_model.torch")